In [1]:
import pandas as pd
import pickle

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from pandas import json_normalize

from sklearn.preprocessing import StandardScaler

from random import randint

In [2]:
with open("model.pkl", "rb") as f:
    kmeans = pickle.load(f)

# Unpickling model

In [3]:
kmeans

KMeans(n_clusters=5)

In [4]:
top100_songs_df = pd.read_csv("top100_songs.csv")
top100_songs_df

,title,artists
0,Fast Car,Luke Combs
1,Last Night,Morgan Wallen
2,Need A Favor,Jelly Roll
3,Take Two,BTS
4,Save Me (with Lainey Wilson),Jelly Roll
...,...,...
95,Memory Lane,Old Dominion
96,A Little Bit Happy,TALK
97,Perfect,Ed Sheeran
98,Whiskey Glasses,Morgan Wallen


In [5]:
clustered_songs_df = pd.read_csv("Clustered_songs.csv")
clustered_songs_df

,Title,Artist,Cluster
0,You're The One - Un mal pour un bien,Petula Clark,1
1,Doctor My Eyes,Jackson Browne,4
2,Falling in Love at a Coffee Shop,Landon Pigg,3
3,Peaceful Easy Feeling - 2013 Remaster,Eagles,4
4,Summertime Blues,Eddie Cochran,0
...,...,...,...
10117,Elements (Deluxe),Ludovico Einaudi,2
10118,Moon Safari,Air,4
10119,Islands - Essential Einaudi,Ludovico Einaudi,2
10120,Liquid Mind II: Slow World,Liquid Mind,2


# Spotify Authentication

In [6]:
secrets_file = open("secrets.txt","r")
string = secrets_file.read()
secret_dict = {}

for line in string.split("\n"):
    if len(line) > 1:
        
        secret_dict[line.split(':')[0]]=line.split(':')[1].strip()
    

In [7]:
auth_manager = SpotifyClientCredentials(client_id = secret_dict['clientid'], 
                                        client_secret = secret_dict['clientsecret'])

sp = spotipy.Spotify(auth_manager = auth_manager)

# Functions for recommendations and predictions

In [8]:
# Pick random songs from the predicted cluster.
def pick_song(cluster):
    
    df = clustered_songs_df[clustered_songs_df.Cluster == cluster].reset_index()
    rand_int = randint(0, len(df))
    return (df.Title[rand_int], df.Artist[rand_int])
    

In [24]:
# Predict cluster of input song
def predict_cluster(audio_features):
    
    X = pd.json_normalize(audio_features[0])
    X = X.drop(["type", "id", "uri", "track_href", "analysis_url", "time_signature"], axis = 1)
    X_scaled = StandardScaler().fit_transform(X)
    
    cluster = kmeans.predict(X_scaled)[0]
    return pick_song(cluster)


In [25]:
# Get audio features of the input song
def get_audio_features(input_song):
    
    results = sp.search(q = input_song, type = "track")
    
    if len(results["tracks"]["items"]) > 1: 
        
        uri = results["tracks"]["items"][0]["uri"]
        audio_features = sp.audio_features(uri)
        return predict_cluster(audio_features)
    else:
        return "No recommendation. Please retry later"    

In [26]:
import random

# Recommend from hot songs
def random_songs(data):
    
    random_number = random.randint(0, len(data) - 1)
    return (data.title[random_number], data.artists[random_number])

# Function to check if hot songs or not
def check_and_recommend(song_name, top100_songs_df):
    
    songs = list(map(str.lower, top100_songs_df.title))
    
    if song_name.lower() in songs:
        
        recommended_song = random_songs(top100_songs_df)
        return "Its hot song!! Check this out as well {0} by {1} ".format(recommended_song[0], recommended_song[1])
    
    else:
        
        recommended_song = get_audio_features(song_name)
        
        if type(recommended_song) == str:
            
            return recommended_song
        else:
            
            st = "Hey do check this song as well,  {0} by {1}".format(recommended_song[0], recommended_song[1])
            return st
    

In [12]:
user_input = input("Type a song title: ")

recommendation = check_and_recommend(user_input, top100_songs_df)
print(recommendation)

Type a song title: take two
Its hot song!! Check this out as well We Don't Fight Anymore (feat. Chris Stapleton) by Carly Pearce 


In [27]:
user_input = input("Type a song title: ")

recommendation = check_and_recommend(user_input, top100_songs_df)
print(recommendation)

Type a song title: Doctor My Eyes
Hey do check this song as well,  The New Sound of the Venezuelan Gozadera by Los Amigos Invisibles


In [28]:
user_input = input("Type a song title: ")

recommendation = check_and_recommend(user_input, top100_songs_df)
print(recommendation)

Type a song title: HJDFSJASDHF
No recommendation. Please retry later


In [29]:
user_input = input("Type a song title: ")

recommendation = check_and_recommend(user_input, top100_songs_df)
print(recommendation)

Type a song title: KESARIYA
Hey do check this song as well,  Animal by Neon Trees
